In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

engine = create_engine('mssql://poseidon/sansys_readonly?trusted_connection=yes&driver=SQL+Server')

In [ ]:
# Função atualiza data
def FormarDataArr(dia_inicial, mes_inicial, ano_inicial, meses_de_relatorio):
    data_inicial = datetime(ano_inicial, mes_inicial, dia_inicial)
    data_final = data_inicial + relativedelta(months = meses_de_relatorio + 1, days=-1)
    data_final_rel = data_inicial + relativedelta(months=1, days=-1)
    while data_final_rel < data_final:
        data_inicial_rel = data_inicial.strftime('%Y%m%d')
        data_final_rel = data_final_rel.strftime('%Y%m%d')
        
        RodarScriptArr(data_inicial_rel, data_final_rel)
        
        data_inicial = data_inicial + relativedelta(months = 1)
        data_final_rel = data_inicial + relativedelta(months=1, days=-1)
        
    print('Fim da geração!')


In [ ]:
# Função para puxar relatório 40 - Relatório de Pagamento de faturas
def RodarScriptArr(data_inicial, data_final):
    inicio = datetime.now()
    print('Iniciada a geração do relatório de ' + data_inicial +' ('+ str(inicio) + ')')
    arr = pd.read_sql_query(f'''  

set transaction isolation level read uncommitted
SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                ff.DT_EMISSAO_FATURA as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'FATURA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                ff.vl_total_fatura as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, ff.dt_mesano_ref) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                ff.dt_vencimento_fatura as "VENCIMENTO",
                tct.NM_CATEGORIA_TARIFA CATEGORIA_PRINCIPAL,
                ff.id_fatura as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                cc.NM_CLIENTE as "Morador",
                Parcelamento.Parcela as "Parcela",
                Parcelamento.Valor as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                ff.vl_total_tbo_agua "TBO ÁGUA",
                ff.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
inner join arr_pagamento_fatura apf on (ap.id_pagamento = apf.id_pagamento)
inner join fat_fatura ff on (apf.id_fatura = ff.id_fatura)
inner join cad_unidade_comercial cuc on (ff.ch_matricula_unidade = cuc.ch_matricula_unidade)
left join TAB_CATEGORIA_TARIFA tct on (ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA)
inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA
                and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE

left join (select ffs.ID_FATURA
                               ,cast(fpsu.NR_PARCELA_SERVICO as varchar) + '/' + cast(fpsu.NR_TOTAL_PARCELA_SERVICO as varchar) as "Parcela"
                               ,fpsu.VL_PARCELA_SERVICO as "Valor"
                from FAT_FATURA_SERVICO ffs
                join FAT_PARCELA_SERVICO_UNIDADE fpsu on fpsu.ID_PARCELA_SERVICO_UNIDADE = ffs.ID_PARCELA_SERVICO_UNIDADE
                join FAT_SERVICO_UNIDADE fsu on fsu.ID_SERVICO_UNIDADE = fpsu.ID_SERVICO_UNIDADE

                join TAB_SERVICO_DEFINICAO tsd on tsd.ID_SERVICO_DEFINICAO = fsu.ID_SERVICO_DEFINICAO
                               and tsd.CD_SERVICO = 1060
) Parcelamento on Parcelamento.ID_FATURA = ff.ID_FATURA

where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                fad.DT_EMISSAO_AVISO_DEBITO as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'COMUNICADO' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fad.vl_total_aviso_debito as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fad.DT_VENCIMENTO_AVISO as "VENCIMENTO",
                categoria.todas CATEGORIA_PRINCIPAL,
                fad.id_aviso_debito as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                total_fatura.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                total_fatura.vl_total_tbo_agua "TBO ÁGUA",
                total_fatura.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_aviso_debito apad on (ap.id_pagamento = apad.id_pagamento)
                inner join fat_aviso_debito fad on (apad.id_aviso_debito = fad.id_aviso_debito)
                inner join cad_unidade_comercial cuc on (fad.ch_matricula_unidade = cuc.ch_matricula_unidade)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join
                               (
                                               select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos,
                                                               SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_AVISO_DEBITO, MAX(cc.NU_CPF_CNPJ) NU_CPF_CNPJ,
                                                               SUM(ISNULL(VL_TOTAL_TBO_AGUA,0)) vl_total_tbo_agua, SUM(ISNULL(vl_total_tbo_esgoto,0)) vl_total_tbo_esgoto
                                               from FAT_AVISO_DEBITO fad
                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fad.ID_AVISO_DEBITO = faf.ID_AVISO_DEBITO)
                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA)
                                                               left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
                                                               left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE
                                               group  by faf.ID_AVISO_DEBITO
                               ) total_fatura on (fad.ID_AVISO_DEBITO = total_fatura.ID_AVISO_DEBITO)
                outer apply
                (
                               select STUFF
                               (
                                               (
                                                               select ', ' + tct.NM_CATEGORIA_TARIFA
                                                               from  FAT_AVISO_DEBITO fad
                                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fad.ID_AVISO_DEBITO = faf.ID_AVISO_DEBITO)
                                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA),
                                                                               TAB_CATEGORIA_TARIFA tct
                                                               where ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA
                                                                               and total_fatura.ID_AVISO_DEBITO = fad.ID_AVISO_DEBITO
                                                               for xml path('')
                                               ), 1, 2, ''
                               ) as [todas]
                ) as categoria
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                fsvf.DT_EMISSAO_SEGUNDA_VIA as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'SEGUNDA VIA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fsvf.vl_total_segunda_via as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                null as "VENCIMENTO",
                categoria.todas ATEGORIA_PRINCIPAL,
                fsvf.id_segunda_via_fatura as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                total_fatura.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                total_fatura.vl_total_tbo_agua "TBO ÁGUA",
                total_fatura.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_segunda_via apsv on (ap.id_pagamento = apsv.id_pagamento)
                inner join fat_segunda_via_fatura fsvf on (apsv.id_segunda_via_fatura = fsvf.id_segunda_via_fatura)
                inner join cad_unidade_comercial cuc on (fsvf.ch_matricula_unidade = cuc.ch_matricula_unidade)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join
                               (
                                               select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos,
                                                               SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_SEGUNDA_VIA_FATURA, MAX(cc.NU_CPF_CNPJ) NU_CPF_CNPJ,
                                                               SUM(ISNULL(VL_TOTAL_TBO_AGUA,0)) vl_total_tbo_agua, SUM(ISNULL(vl_total_tbo_esgoto,0)) vl_total_tbo_esgoto
                                               from fat_segunda_via_fatura fsvf
                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsvf.ID_SEGUNDA_VIA_FATURA = faf.ID_SEGUNDA_VIA_FATURA)
                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA)
                                                               left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
                                                               left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE
                                               group  by faf.ID_SEGUNDA_VIA_FATURA
                               ) total_fatura on (fsvf.ID_SEGUNDA_VIA_FATURA = total_fatura.ID_SEGUNDA_VIA_FATURA)
                outer apply
                (
                               select STUFF
                               (
                                               (
                                                               select ', ' + tct.NM_CATEGORIA_TARIFA
                                                               from  fat_segunda_via_fatura fsvf
                                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsvf.ID_SEGUNDA_VIA_FATURA = faf.ID_SEGUNDA_VIA_FATURA)
                                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA),
                                                                               TAB_CATEGORIA_TARIFA tct
                                                               where ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA
                                                                               and total_fatura.ID_SEGUNDA_VIA_FATURA = fsvf.ID_SEGUNDA_VIA_FATURA
                                                               for xml path('')
                                               ), 1, 2, ''
                               ) as [todas]
                ) as categoria

                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                fcd.DT_EMISSAO_COBRANCA_DIVERSA as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'COBRANÇA DIVERSA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fcd.vl_total_cobranca_diversa as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fcd.DT_VENCIMENTO_COBRANCA_DIVERSA as "VENCIMENTO",
		NULL CATEGORIA_PRINCIPAL,
                fcd.id_cobranca_diversa as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_cobranca_diversa apcd on (ap.id_pagamento = apcd.id_pagamento)
                inner join fat_cobranca_diversa fcd on (apcd.id_cobranca_diversa = fcd.id_cobranca_diversa)
                inner join cad_unidade_comercial cuc on (fcd.ch_matricula_unidade = cuc.ch_matricula_unidade)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = fcd.CH_CLIENTE
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                fcd.DT_EMISSAO_COBRANCA_DIVERSA as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                cast(ccc.ch_matricula_cliente_comercial as varchar(10)) + '-' + ccc.digito_matricula_cliente_comercial as "MATRÍCULA",
                '' GRUPO,
                'COBRANÇA DIVERSA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fcd.vl_total_cobranca_diversa as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fcd.DT_VENCIMENTO_COBRANCA_DIVERSA as "VENCIMENTO",
		NULL CATEGORIA_PRINCIPAL,
                fcd.id_cobranca_diversa as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_cobranca_diversa apcd on (ap.id_pagamento = apcd.id_pagamento)
                inner join fat_cobranca_diversa fcd on (apcd.id_cobranca_diversa = fcd.id_cobranca_diversa)
                inner join cad_cliente_comercial ccc on (fcd.ch_matricula_cliente_comercial = ccc.ch_matricula_cliente_comercial)
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = fcd.CH_CLIENTE
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                fsfc.DT_EMISSAO_FATURA_CENTRALIZADA as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                cast(cuc.CD_ORGAO_CENTRALIZADOR as varchar(10)) as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'FATURA CENTRALIZADA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fsfc.VL_TOTAL_FATURA_CENTRALIZADA as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fsfc.DT_VENCIMENTO_FATURA_CENTRALIZADA as "VENCIMENTO",
                categoria.todas CATEGORIA_PRINCIPAL,
                fsfc.ID_FATURA_CENTRALIZADA as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                total_fatura.vl_total_tbo_agua "TBO ÁGUA",
                total_fatura.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join ARR_PAGAMENTO_FATURA_CENTRALIZADA apfc on (ap.id_pagamento = apfc.id_pagamento)
                inner join FAT_FATURA_CENTRALIZADA fsfc on (apfc.ID_FATURA_CENTRALIZADA = fsfc.ID_FATURA_CENTRALIZADA)
                inner join CAD_ORGAO_CENTRALIZADOR cuc on (fsfc.ID_ORGAO_CENTRALIZADOR = cuc.ID_ORGAO_CENTRALIZADOR)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join
                               (
                                               select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos,
                                                               SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_FATURA_CENTRALIZADA,
                                                               SUM(ISNULL(VL_TOTAL_TBO_AGUA,0)) vl_total_tbo_agua, SUM(ISNULL(vl_total_tbo_esgoto,0)) vl_total_tbo_esgoto
                                               from FAT_FATURA_CENTRALIZADA fsfc
                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsfc.ID_FATURA_CENTRALIZADA = faf.ID_FATURA_CENTRALIZADA)
                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA)
                                               group  by faf.ID_FATURA_CENTRALIZADA
                                ) total_fatura on (fsfc.ID_FATURA_CENTRALIZADA = total_fatura.ID_FATURA_CENTRALIZADA)
                outer apply
                (
                               select STUFF
                               (
                                               (
                                                               select ', ' + tct.NM_CATEGORIA_TARIFA
                                                               from  FAT_FATURA_CENTRALIZADA fsfc
                                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsfc.ID_FATURA_CENTRALIZADA = faf.ID_FATURA_CENTRALIZADA)
                                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA),
                                                                               TAB_CATEGORIA_TARIFA tct
                                                               where ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA
                                                                               and total_fatura.ID_FATURA_CENTRALIZADA = fsfc.ID_FATURA_CENTRALIZADA
                                                               for xml path('')
                                               ), 1, 2, ''
                               ) as [todas]
                ) as categoria
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = cuc.CH_CLIENTE
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                aaf.DT_PROCESSAMENTO_SISTEMA as "DATA PROCESSAMENTO SISTEMA",
                null as "DATA EMISSÃO FATURA",
                ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                '' as "MATRÍCULA",
                '' as GRUPO,
                'NAO IDENTIFICADO' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                ap.vl_valor_debito as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                ap.DT_VENCIMENTO_DEBITO as "VENCIMENTO",
		NULL CATEGORIA_PRINCIPAL,
                0 as "ID DO DOCUMENTO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                '' as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) and (ap.ch_situacao_pagto = 2)
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

select taa.NM_AGENTE_ARRECADADOR as "AGENTE ARRECADADOR",
                NULL as "CONVENIO",
                adb.NUMERO_DOCUMENTO_DEPOSITO as "NSA",
                adb.DT_INCLUSAO as "DATA PROCESSAMENTO SISTEMA",
                null as "DATA EMISSÃO FATURA",
                adb.DT_DEPOSITO as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                NULL as "MATRÍCULA",
                NULL as GRUPO,
                'NAO IDENTIFICADO' as "TIPO DE DOCUMENTO",
                adb.VL_DEPOSITADO as "VALOR",
                NULL as "VALOR DOCUMENTO",
                NULL as "REFERÊNCIA DA FATURA",
                NULL as "VENCIMENTO",
                NULL as "CATEGORIA_PRINCIPAL",
                NULL as "ID DO DOCUMENTO",
                NULL as "FORMA PAGTO",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                '' as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from ARR_DEPOSITO_BANCARIO adb
                inner join TAB_AGENCIA ta on ta.ID_AGENCIA = adb.ID_AGENCIA
                inner join TAB_AGENTE_ARRECADADOR taa on taa.ID_AGENTE_ARRECADADOR = ta.ID_AGENTE_ARRECADADOR
                left join ARR_PAGAMENTO ap on ap.ID_DEPOSITO_BANCARIO = adb.ID_DEPOSITO_BANCARIO
where adb.FL_DEPOSITO_IDENTIFICADO = 'N'
                and adb.DT_DEPOSITO between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/
                and not exists (select ID_DEPOSITO_BANCARIO from ARR_PAGAMENTO where ID_DEPOSITO_BANCARIO = adb.ID_DEPOSITO_BANCARIO)
                
    ''', engine)
    arr.to_csv('bd_arr/'+data_inicial + 'arr.csv', sep=';', encoding='utf-8')
    fim = datetime.now()
    print('Relatório ' + data_inicial + 'arr.csv salvo! Gerado em ' + str(fim - inicio))        

In [ ]:
dia = int(input('Dia inicial: '))
mes = int(input('Mês inicial: '))
ano = int(input('Ano inicial: '))
qtd_meses = int(input('Quantidade de meses: '))

FormarDataArr(dia, mes, ano, qtd_meses)

In [ ]:
arr_geral = pd.DataFrame()

In [ ]:
%%time
arr_2005_2010 = pd.DataFrame()

colunas_a_remover =['Unnamed: 0','ID ARQUIVO', 'ID PAGAMENTO','VALOR ÁGUA', 'VALOR ESGOTO',
'VALOR SERVIÇOS', 'CONTEUDO ARQUIVO','TARIFA', 'Morador', 
'Parcela', 'Valor','CPF/CNPJ Responsável', 'TBO ÁGUA', 'TBO ESGOTO']

lista_arquivos = os.listdir('C:/Users/thiago.cunha/Documents/Python/bd_arr/2005-2010')
for arquivo in lista_arquivos:
    rel_arr = pd.read_csv('bd_arr/2005-2010/' + arquivo, sep=';', encoding='utf-8', low_memory=False)
    rel_arr.drop(columns = colunas_a_remover, inplace=True)
    arr_2005_2010 = pd.concat([arr_2005_2010, rel_arr], axis=0, ignore_index=True)



In [ ]:
%%time

arr_2011_2014 = pd.DataFrame()

colunas_a_remover =['Unnamed: 0','ID ARQUIVO', 'ID PAGAMENTO','VALOR ÁGUA', 'VALOR ESGOTO',
'VALOR SERVIÇOS', 'CONTEUDO ARQUIVO','TARIFA', 'Morador', 
'Parcela', 'Valor','CPF/CNPJ Responsável', 'TBO ÁGUA', 'TBO ESGOTO']

lista_arquivos = os.listdir('C:/Users/thiago.cunha/Documents/Python/bd_arr/2011-2014')
for arquivo in lista_arquivos:
    rel_arr = pd.read_csv('bd_arr/2011-2014/' + arquivo, sep=';', encoding='utf-8', low_memory=False)
    rel_arr.drop(columns = colunas_a_remover, inplace=True)
    arr_2011_2014 = pd.concat([arr_2011_2014, rel_arr], axis=0, ignore_index=True)



In [ ]:
%%time

arr_2015_2018 = pd.DataFrame()

colunas_a_remover =['Unnamed: 0','ID ARQUIVO', 'ID PAGAMENTO','VALOR ÁGUA', 'VALOR ESGOTO',
'VALOR SERVIÇOS', 'CONTEUDO ARQUIVO','TARIFA', 'Morador', 
'Parcela', 'Valor','CPF/CNPJ Responsável', 'TBO ÁGUA', 'TBO ESGOTO']

lista_arquivos = os.listdir('C:/Users/thiago.cunha/Documents/Python/bd_arr/2015-2018')
for arquivo in lista_arquivos:
    rel_arr = pd.read_csv('bd_arr/2015-2018/' + arquivo, sep=';', encoding='utf-8', low_memory=False)
    rel_arr.drop(columns = colunas_a_remover, inplace=True)
    arr_2015_2018 = pd.concat([arr_2015_2018, rel_arr], axis=0, ignore_index=True)



In [ ]:
%%time

arr_2019_2021 = pd.DataFrame()

colunas_a_remover =['Unnamed: 0','ID ARQUIVO', 'ID PAGAMENTO','VALOR ÁGUA', 'VALOR ESGOTO',
'VALOR SERVIÇOS', 'CONTEUDO ARQUIVO','TARIFA', 'Morador', 
'Parcela', 'Valor','CPF/CNPJ Responsável', 'TBO ÁGUA', 'TBO ESGOTO']

lista_arquivos = os.listdir('C:/Users/thiago.cunha/Documents/Python/bd_arr/2019-2021')
for arquivo in lista_arquivos:
    rel_arr = pd.read_csv('bd_arr/2019-2021/' + arquivo, sep=';', encoding='utf-8', low_memory=False)
    rel_arr.drop(columns = colunas_a_remover, inplace=True)
    arr_2019_2021 = pd.concat([arr_2019_2021, rel_arr], axis=0, ignore_index=True)



In [ ]:
%%time
arr_2005_2010.to_csv('bd_arr/arr_2005_2010.csv', sep=';', encoding='utf-8', index=False)
arr_2011_2014.to_csv('bd_arr/arr_2011_2014.csv', sep=';', encoding='utf-8', index=False)
arr_2015_2018.to_csv('bd_arr/arr_2015_2018.csv', sep=';', encoding='utf-8', index=False)
arr_2019_2021.to_csv('bd_arr/arr_2019_2021.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
arr_geral = pd.DataFrame()
arr_geral = pd.concat([arr_geral, arr_2005_2010], axis=0, ignore_index=True)
arr_geral = pd.concat([arr_geral, arr_2011_2014], axis=0, ignore_index=True)
arr_geral = pd.concat([arr_geral, arr_2015_2018], axis=0, ignore_index=True)
arr_geral = pd.concat([arr_geral, arr_2019_2021], axis=0, ignore_index=True)

In [ ]:
arr_2015_2018.to_json('bd_arr/arr_2015_2018.json', orient='table', index=False)

In [ ]:
arr_2019_2021.head()

In [ ]:
%%time
arr_geral = pd.concat([arr_2005_2010, arr_2011_2014, arr_2015_2018, arr_2019_2021], axis=0, ignore_index=True)

In [ ]:
arr_geral['DATA PAGAMENTO'].max()

In [ ]:
arr_geral['DATA PAGAMENTO'].min()

In [ ]:
%%time
arr_geral.to_csv('bd_arr/arr_geral.csv', sep=';', encoding='utf-8', index=False)